In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install peft
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/only-halue/my_model_final-llama-onlyhalueval/adapter_model.safetensors
/kaggle/input/only-halue/my_model_final-llama-onlyhalueval/training_args.bin
/kaggle/input/only-halue/my_model_final-llama-onlyhalueval/adapter_config.json
/kaggle/input/only-halue/my_model_final-llama-onlyhalueval/README.md
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 8.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import re
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, XLNetTokenizerFast,LlamaModel,LlamaTokenizerFast,PreTrainedTokenizerFast
import huggingface_hub 
from transformers import LlamaForCausalLM
from transformers import Trainer, TrainingArguments

import json
from huggingface_hub import login
import torch
from peft import LoraConfig,get_peft_model
import wandb

lora_config = LoraConfig(
    r = 16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="TOKEN_CLS"     
)




tokenizer = LlamaTokenizerFast.from_pretrained('meta-llama/Llama-3.2-1B')
tokenizer.pad_token = tokenizer.decode(128004)
ds = load_dataset('wandb/RAGTruth-processed')
ds = ds.filter(lambda example: example['task_type'] == 'QA')

dataset = ds['train']

label=[]

def preprocess_data(data):
    end_token = tokenizer.decode(128001)
    start_token = tokenizer.decode(128000)

    
    input_text = f"{data.get('query')}{end_token}{start_token}{data.get('context')}{end_token}"
    text_pair = f"{data.get('output')}"

    
    a = data['hallucination_labels']
    a = json.loads(a)
    start = []
    end = []
    for item in a:
        start.append(item['start'])
        end.append(item['end'])

    
    inputs = tokenizer(text=input_text, text_pair=text_pair, padding='max_length', max_length=1024, return_tensors='pt',
                       return_offsets_mapping=True,truncation=True)

    
    labels = [-100] * len(inputs['input_ids'][0])
    
    start_flag_index = None
    end_flag_index = None
    
    start_flag = inputs.encodings[0].sequence_ids
    b = True
    for i in range(0, len(start_flag)):
        if start_flag[i] == 1:
            labels[i] = 3
            end_flag_index = i
            if b:
                start_flag_index = i
                b = False

    
    offset_mapping = inputs.encodings[0].offsets
    inputs.pop('offset_mapping')
    
    for i in range(len(start)):
        tmp = True
        for j in range(start_flag_index, end_flag_index+1):
            
            a = offset_mapping[j]
            if (offset_mapping[j][0] >= start[i] or offset_mapping[j][0] == start[i] - 1) and offset_mapping[j][1] <= \
                    end[i]:
                if tmp:
                    labels[j] = 0
                    tmp = False
                else:
                    labels[j] = 1
                    if offset_mapping[j][1] == end[i]:
                        labels[j] = 2
                        break

    
    long_tensor = torch.tensor(labels, dtype=torch.long)
    inputs['input_ids']=inputs['input_ids'][0]
    inputs['attention_mask']=inputs['attention_mask'][0]
    inputs['labels'] = long_tensor
    return inputs

dataset = dataset.map(preprocess_data,batched=False)
dataset = dataset.remove_columns(['id', 'query','context','output', 'hallucination_labels', 'hallucination_labels_processed', 'input_str','task_type', 'quality', 'model', 'temperature'])



In [3]:


id2label = {
    0: 'B-HAL',  
    1: 'I-HAL',  
    2: 'E-HAL', 
    3: 'O'       
}
label2id = {
    'B-HAL': 0,
    'I-HAL': 1,
    'E-HAL': 2,
    'O': 3
}
from transformers import LlamaForTokenClassification,AutoModelForTokenClassification
from peft import PeftModel

model = LlamaForTokenClassification.from_pretrained('meta-llama/Llama-3.2-1B', num_labels=4, id2label=id2label, label2id=label2id,output_hidden_states=True)

lora_model = PeftModel.from_pretrained(model,"/kaggle/input/only-halue/my_model_final-llama-onlyhalueval")



config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
trainset, valset = dataset.train_test_split(test_size=0.2).values()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=valset,
)


trainer.train()

In [ ]:
trainer.save_model("./my_model_final-llama-halurag") 